In [1]:
import numpy as np 
import pandas as pd 
from umap import UMAP 
import leidenalg
import igraph as ig
import math
from sklearn.base import BaseEstimator
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors


/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def leiden_cluster(X, k=25):
    """
    Create a graph from nearest neighbors and find clusters using Leiden algorithm
    
    Parameters:
    -----------
    X : array-like
        The dimensionality reduced data
    k : int, default=25
        Number of nearest neighbors
        
    Returns:
    --------
    g : igraph.Graph
        The created graph
    partition : leidenalg.VertexPartition
        The partition result from Leiden algorithm
    """
    # Find nearest neighbors
    neighbors = NearestNeighbors(n_neighbors=k).fit(X)
    distances, indices = neighbors.kneighbors(X)

    # Build edge list with weighted edges
    edges = []
    weights = []
    num_points = X.shape[0]

    for i in range(num_points):
        for idx, j in enumerate(indices[i]):
            if i == j: 
                continue
            if (j, i) in edges:
                continue
            d = distances[i, idx]
            weight = math.exp(-d)
            edges.append((i, j))
            weights.append(weight)

    # Create an igraph Graph, add vertices and edges
    g = ig.Graph()
    g.add_vertices(num_points)
    g.add_edges(edges)

    # Set the edge attribute 'weight' for our weighted graph
    g.es['weight'] = weights

    # Find partition using Leiden algorithm
    partition = leidenalg.find_partition(g, leidenalg.ModularityVertexPartition)
    print("Clusters:", partition)
    
    return g, partition


In [6]:
class UMAPClusteringEvaluator(BaseEstimator):
    def __init__(self, n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean'):
        """
        Initialize UMAP and store hyperparameters for later grid search.
        """
        self.n_neighbors = n_neighbors
        self.min_dist = min_dist
        self.n_components = n_components
        self.metric = metric
        self.umap_model = None

    def fit(self, X, y=None):
        """
        Fit the UMAP model on the data.
        """
        self.umap_model = UMAP(n_neighbors=self.n_neighbors,
                                    min_dist=self.min_dist,
                                    n_components=self.n_components,
                                    metric=self.metric,
                                    random_state=42)
        self.umap_model.fit(X)
        return self

    def transform(self, X):
        """
        Transform the data with the fitted UMAP model.
        """
        return self.umap_model.transform(X)

    def score(self, X, y=None):
        """
        Transform the data into the reduced space and then run a set of clustering algorithms.
        Compute the silhouette score for each (if valid) and return the average silhouette score.
        """
        X_reduced = self.transform(X)
        scores = []  # To collect silhouette scores
        
        # List of clustering algorithms to evaluate.
        # You can adjust these or add new ones as needed.
        clustering_methods = [
            ('KMeans', KMeans()), # 6 k means works the best 
            ('Agglomerative', AgglomerativeClustering()),
            ('Spectral', SpectralClustering()),
            ('DBSCAN', DBSCAN()),
            ('GaussianMixture', GaussianMixture()),
            ('Leiden', leiden_cluster(X_reduced)) # Custom Leiden clustering,
        ]
        
        for name, algorithm in clustering_methods:
            try:
                # Obtain cluster labels.
                # Some algorithms have fit_predict, others require separate fitting and predicting.
                if name in ['KMeans', 'Agglomerative', 'Spectral', 'DBSCAN', 'OPTICS']:
                    labels = algorithm.fit_predict(X_reduced)
                elif name == 'GaussianMixture':
                    algorithm.fit(X_reduced)
                    labels = algorithm.predict(X_reduced)
                elif name == 'Leiden':
                    g, partition = leiden_cluster(X_reduced)
                    labels = np.array(partition.membership)
                
                # Check if we have at least two clusters.
                # For DBSCAN, we exclude noise labeled as -1.
                if name == 'DBSCAN':
                    valid_idx = labels != -1
                    if len(np.unique(labels[valid_idx])) < 2:
                        continue
                    score = silhouette_score(X_reduced[valid_idx], labels[valid_idx])
                else:
                    if len(np.unique(labels)) < 2:
                        continue  # Skip if only one cluster is produced.
                    score = silhouette_score(X_reduced, labels)
                
                scores.append(score)
            except Exception as e:
                # In a production system you might log errors; here we just print them.
                print(f"Error with {name}: {e}")
                continue

        # If none of the clustering methods produced a valid silhouette score,
        # return a default low score. Otherwise, return the average.
        if scores:
            return np.mean(scores)
        return -1.0  # or another value indicating failure


In [7]:
df_log = pd.read_csv('/Users/harrisonma/Documents/PhD Classes/2025 Spring/BNFO 285/BNFO285_Projects/project_1/Harrison/Harrison_Local_Data/TCGA.HNSC.expression_log_all.txt', sep = '\t')

np_log = df_log.to_numpy()
np_log = np.delete(np_log, [0, 1], axis=1)


In [8]:
X = np_log

# Define parameter grid for UMAP hyperparameters.
param_grid = {
    'n_neighbors': [10, 15, 20],
    'min_dist': [0.1, 0.5],
    'n_components': [2, 5, 10]
}

# Set up GridSearchCV. Note that we don't need to specify a separate scoring function,
# because the custom estimator's score() method is used by default.
grid_search = GridSearchCV(estimator=UMAPClusteringEvaluator(),
                           param_grid=param_grid,
                           cv=5,  # 3-fold cross-validation; adjust as needed.
                           n_jobs=-1,  # Use all available cores.
                           verbose=1)

# Run the grid search over the UMAP hyperparameters.
grid_search.fit(X)

# Report the best hyperparameters and corresponding silhouette score.
print("Best parameters:", grid_search.best_params_)
print("Best average silhouette score:", grid_search.best_score_)


Fitting 5 folds for each of 18 candidates, totalling 90 fits


/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Clusters: Clustering with 109 elements and 3 clusters
[0] 4, 5, 9, 10, 14, 16, 18, 22, 23, 24, 30, 34, 35, 36, 37, 39, 41, 42, 44,
    47, 48, 55, 57, 58, 59, 62, 65, 66, 68, 71, 74, 84, 85, 87, 94, 96, 97,
    98, 100, 104, 106
[1] 0, 12, 20, 28, 31, 43, 52, 53, 54, 56, 60, 63, 64, 67, 69, 70, 72, 73, 76,
    77, 79, 80, 81, 82, 83, 88, 89, 90, 91, 92, 93, 95, 99, 101, 102, 103
[2] 1, 2, 3, 6, 7, 8, 11, 13, 15, 17, 19, 21, 25, 26, 27, 29, 32, 33, 38, 40,
    45, 46, 49, 50, 51, 61, 75, 78, 86, 105, 107, 108
Clusters: Clustering with 109 elements and 3 clusters
[0] 0, 2, 6, 12, 20, 28, 31, 43, 52, 53, 54, 56, 60, 63, 64, 67, 69, 70, 72,
    73, 75, 76, 77, 79, 80, 81, 82, 83, 88, 89, 90, 91, 92, 93, 95, 99, 101,
    102, 103, 105
[1] 1, 3, 7, 8, 11, 13, 15, 16, 17, 19, 21, 22, 25, 26, 27, 29, 30, 32, 33,
    34, 38, 40, 45, 46, 47, 49, 50, 51, 55, 57, 58, 61, 78, 86, 98, 107, 108
[2] 4, 5, 9, 10, 14, 18, 23, 24, 35, 36, 37, 39, 41, 42, 44, 48, 59, 62, 65,
    66, 68, 71, 74, 84, 85, 87

/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/sklearn/utils/depre

Clusters: Clustering with 109 elements and 3 clusters
[0] 2, 3, 4, 6, 7, 9, 10, 11, 12, 13, 22, 32, 34, 44, 50, 51, 53, 54, 55, 56,
    57, 59, 60, 64, 66, 68, 71, 73, 76, 78, 79, 82, 89, 91, 92, 99, 100, 102,
    103, 104
[1] 0, 16, 18, 19, 20, 23, 26, 28, 29, 31, 33, 35, 41, 42, 43, 61, 62, 63, 65,
    67, 69, 74, 75, 80, 83, 84, 85, 87, 88, 90, 94, 95, 96, 98, 101, 105, 106,
    107
[2] 1, 5, 8, 14, 15, 17, 21, 24, 25, 27, 30, 36, 37, 38, 39, 40, 45, 46, 47,
    48, 49, 52, 58, 70, 72, 77, 81, 86, 93, 97, 108
Clusters: Clustering with 109 elements and 3 clusters
[0] 2, 3, 4, 6, 7, 9, 10, 11, 12, 13, 22, 32, 34, 44, 50, 51, 53, 54, 55, 56,
    57, 59, 60, 64, 66, 68, 71, 73, 76, 78, 79, 82, 89, 91, 92, 99, 100, 102,
    103, 104
[1] 0, 16, 18, 19, 20, 23, 26, 28, 29, 31, 33, 35, 41, 42, 43, 61, 62, 63, 65,
    67, 69, 74, 75, 80, 83, 84, 85, 87, 88, 90, 94, 95, 96, 98, 101, 105, 106,
    107
[2] 1, 5, 8, 14, 15, 17, 21, 24, 25, 27, 30, 36, 37, 38, 39, 40, 45, 46, 47,
    48, 49, 52, 

...

Clusters: Clustering with 109 elements and 3 clusters
[0] 8, 9, 10, 12, 13, 16, 23, 27, 28, 29, 33, 35, 40, 44, 45, 46, 47, 48, 51,
    52, 54, 55, 58, 59, 62, 63, 64, 73, 75, 76, 78, 79, 80, 86, 93, 95, 96,
    97, 101, 102, 103, 104, 105
[1] 0, 4, 5, 11, 15, 18, 21, 22, 26, 30, 31, 41, 42, 43, 49, 50, 53, 57, 61,
    66, 69, 70, 71, 74, 81, 82, 84, 85, 88, 92, 94, 98, 99, 100, 106, 107
[2] 1, 2, 3, 6, 7, 14, 17, 19, 20, 24, 25, 32, 34, 36, 37, 38, 39, 56, 60, 65,
    67, 68, 72, 77, 83, 87, 89, 90, 91, 108
Clusters: Clustering with 109 elements and 3 clusters
[0] 1, 2, 3, 6, 7, 14, 17, 19, 20, 23, 24, 25, 32, 34, 36, 37, 38, 39, 56, 59,
    60, 64, 65, 67, 68, 72, 76, 77, 80, 83, 87, 89, 90, 91, 95, 102, 108
[1] 8, 9, 10, 12, 13, 16, 27, 28, 29, 33, 35, 40, 44, 45, 46, 47, 48, 51, 52,
    54, 55, 58, 62, 63, 73, 75, 78, 79, 86, 93, 96, 97, 101, 103, 104, 105
[2] 0, 4, 5, 11, 15, 18, 21, 22, 26, 30, 31, 41, 42, 43, 49, 50, 53, 57, 61,
    66, 69, 70, 71, 74, 81, 82, 84, 85, 88, 92, 94

/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Clusters: Clustering with 109 elements and 3 clusters
[0] 1, 5, 8, 11, 12, 16, 18, 20, 21, 22, 28, 30, 34, 35, 36, 38, 42, 44, 49,
    50, 51, 52, 53, 54, 56, 58, 61, 62, 63, 64, 77, 78, 80, 82, 95, 100, 101,
    102, 103, 104
[1] 0, 3, 13, 24, 27, 29, 31, 32, 37, 40, 41, 43, 46, 47, 48, 55, 60, 65, 66,
    67, 68, 69, 71, 72, 73, 76, 79, 81, 83, 84, 85, 86, 87, 89, 90, 91, 92,
    94, 97, 105
[2] 2, 4, 6, 7, 9, 10, 14, 15, 17, 19, 23, 25, 26, 33, 39, 45, 57, 59, 70, 74,
    75, 88, 93, 96, 98, 99, 106, 107, 108
Clusters: Clustering with 109 elements and 3 clusters
[0] 1, 5, 8, 11, 12, 16, 18, 20, 21, 22, 28, 30, 34, 35, 36, 38, 42, 44, 49,
    50, 51, 52, 53, 54, 56, 58, 61, 62, 63, 64, 77, 78, 80, 82, 95, 100, 101,
    102, 103, 104
[1] 0, 3, 13, 24, 27, 29, 31, 32, 37, 40, 41, 43, 46, 47, 48, 55, 60, 65, 66,
    67, 68, 69, 71, 72, 73, 76, 79, 81, 83, 84, 85, 86, 87, 89, 90, 91, 92,
    94, 97, 105
[2] 2, 4, 6, 7, 9, 10, 14, 15, 17, 19, 23, 25, 26, 33, 39, 45, 57, 59, 70, 74,
    75

/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/harrisonma/opt/anaconda3/envs/ai_cellchat/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Best parameters: {'min_dist': 0.5, 'n_components': 2, 'n_neighbors': 20}
Best average silhouette score: 0.49163851141929626
